In [1]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

# How to Evaluate Propositional Formulas

In this section, we want to show how to evaluate formulas from *propositional logic*.  First of all we need to discuss how to represent logical formulas.  It is easiest if logical formula are represented as nested tuples.  Furthermore, we will represent the propositional operators as unicode symbols according to the following table:
<table border="1px">
    <tr>
        <th>Name</th>
        <th>Symbol</th>
        <th>Unicode</th>
        <th>Unicode Name</th>
    </tr>
    <tr>
        <td><tt>and</tt></td>
        <td>∧</td>
        <td>U+2227</td>
        <td>'\N{logical and}'</td>
    </tr>
    <tr>
        <td><tt>or</tt></td>
        <td>∨</td>
        <td>U+2228</td>
        <td>'\N{logical or}'</td>
    </tr>
    <tr>
        <td><tt>not</tt></td>
        <td>¬</td>
        <td>U+00ac</td>
        <td>'\N{not sign}'</td>
    </tr>
    <tr>
        <td><tt>implies</tt></td>
        <td>→</td>
        <td>U+2192</td>
        <td>'\N{rightwards arrow}'</td>
    </tr>
    <tr>
        <td><tt>if and only if</tt></td>
        <td>↔</td>
        <td>U+2194</td>
        <td>'\N{left right arrow}'</td>
    </tr>
    <tr>
        <td><tt>falsum</tt></td>
        <td>⊥</td>
        <td>U+22a5</td>
        <td>'\N{down tack}'</td>
    </tr>
    <tr>
        <td><tt>verum</tt></td>
        <td>⊤</td>
        <td>U+22a4</td>
        <td>'\N{up tack}'</td>
    </tr>
</table>
For example, the formula 
$$ (p \rightarrow q) \wedge (\neg p \rightarrow q) \rightarrow q $$
will be represented as the following tuple:

```
      ('→', ('∧', ('→', 'p', 'q'), ('→', ('¬', 'p'), 'q')), 'q')
```

As it is very tedious to type formulas this way, I have written a parser for propositional formulas, which we will import.

In [2]:
import propLogParser as plp

Let us test this parser:

In [3]:
plp.LogicParser('⊤').parse()

('⊤',)

In [4]:
s = '(p → q) ∧ (¬p → q) → q'
f = plp.LogicParser(s).parse()
f

('→', ('∧', ('→', 'p', 'q'), ('→', ('¬', 'p'), 'q')), 'q')

Next, we need to decide how to represent a *propositional valuation*.  Mathematically, a *propositional valuation* $\mathcal{I}$ is a function of the form
$$ \mathcal{I}:\mathcal{P} \rightarrow \mathbb{B}  $$
that maps propositional variables to the set of Boolean values 
$\mathbb{B} = \{\texttt{True}, \texttt{False}\}$.  We can represent a propositional  valuation $\mathcal{I}$ as the set of those variables that are evaluated as <tt>True</tt> by $\mathcal{I}$:  
$$ \texttt{repr}(\mathcal{I}) := \bigl\{ v \in \mathcal{P} \mid \mathcal{I}(v) = \texttt{True} \bigr\}$$
For example, the following set would be a propositional valuation for the formula $\texttt{f}$:

In [5]:
I1 = { 'p' , 'q' }

There are 4 possible propositional valuation for the formula $\texttt{f}$.  The other three propositional valuations are as follows:

In [6]:
I2 = { 'p' }
I3 = { 'q' }
I4 = set()

The function `evaluate` defined below takes two arguments:
   - `F` is a nested tuple representing a propositional formula.
   - `I` is a set of propositional variables representing a propositional valuation.
   
The function returns the value of the propositional formula `F` given the propositional valuation `I`.
When evaluation a formula of the form $f \rightarrow g$ this function uses the equivalence
$$ (f \rightarrow g) \;\Leftrightarrow\; \neg f \vee g $$ 

In [7]:
def evaluate(F, I):
    'Evaluate the propositional formula F using the interpretation I'
    if isinstance(F, str):       # F is a propositional variable
        return F in I            # This variable is true iff it occurs in I
    if F[0] == '⊤': return True
    if F[0] == '⊥': return False
    if F[0] == '¬': return not evaluate(F[1], I)
    if F[0] == '∧': return evaluate(F[1], I) and evaluate(F[2], I)
    if F[0] == '∨': return evaluate(F[1], I) or evaluate(F[2], I)
    if F[0] == '→': return not evaluate(F[1], I) or evaluate(F[2], I)
    if F[0] == '↔': return evaluate(F[1], I) == evaluate(F[2], I)

Let's test this function for the formula <tt>f</tt> defined above:

In [8]:
for I in [I1, I2, I3, I4]:
    print(f'evaluate({f}, {I}) = {evaluate(f, I)}')

evaluate(('→', ('∧', ('→', 'p', 'q'), ('→', ('¬', 'p'), 'q')), 'q'), {'q', 'p'}) = True
evaluate(('→', ('∧', ('→', 'p', 'q'), ('→', ('¬', 'p'), 'q')), 'q'), {'p'}) = True
evaluate(('→', ('∧', ('→', 'p', 'q'), ('→', ('¬', 'p'), 'q')), 'q'), {'q'}) = True
evaluate(('→', ('∧', ('→', 'p', 'q'), ('→', ('¬', 'p'), 'q')), 'q'), set()) = True


As the formula <tt>f</tt> is true for **every** possible propositional valuation `I`, it is a *tautology*.